In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [7]:
## load the ann trained model, scaler pickle and onehot

model=load_model('model.h5')

## load the encoder and scaler

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)


with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
     scaler=pickle.load(file)

In [8]:
# Example input data

input_data = {
    'Credit Score': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 30,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}



In [13]:
# onehot encode 'geography'

geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\chait\OneDrive\Documents\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df=pd.DataFrame([input_data])
input_df

,Credit Score,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,60000,2,1,1,50000


In [27]:
## encode the categorical variables

input_df['Gender']=label_encoder_gender.fit_transform(input_df['Gender'])
input_df

,Credit Score,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,30,60000,2,1,1,50000


In [28]:
## concatiination with onhot encoded 

input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,Credit Score,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,30,60000,2,1,1,50000,1.0,0.0,0.0


In [37]:
## scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [40]:
## prediction churn

prediction=model.predict(input_scaled)
prediction



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


array([[0.17758965]], dtype=float32)

In [41]:
prediction_proba=prediction[0][0]

In [42]:
prediction_proba

np.float32(0.17758965)

In [43]:
if prediction_proba > 0.5:
    print('the persion is likely to churn')
else:
    print('the customer not likely to churn')

the customer not likely to churn
